## 아마존 가격 추적기 만들기 
- 자주 변경되는 아마존 가격을 추적하자!
    - 매일 아침 9시에 입력한 url을 확인 
    - 스크래핑을 하여 가장 낮은 price를 검색
    - 해당 결과를 메일로 보내기!

### 1. BeautyfulSoup으로 상품 가겨 스크래핑 하기 
-  아마존에서 가격을 추적하고 싶은 상품을 찾아 상품 URL을 가져오기. 
    - 대상 URL: https://www.amazon.com/-/ko/dp/B08JXG2518/ref=d_pb_allspark_dp_sims_pao_desktop_session_based_sccl_3_4/137-4205210-2076769?pd_rd_w=YOpxg&content-id=amzn1.sym.6b5008ac-c24a-4aea-a3ea-015a531184f5&pf_rd_p=6b5008ac-c24a-4aea-a3ea-015a531184f5&pf_rd_r=P8KNZZ318D1F1YV7G1C5&pd_rd_wg=D2u0A&pd_rd_r=709f08ca-e3ba-456d-bbc4-8a15fac78a5a&pd_rd_i=B08JXG2518&psc=1
    - 브라우저는 아마존에서 페이지를 불러올 때, URL뿐만 아니라 다른 정보도 많이 전달함. 예) 사용하고 있는 브라우저 종류, 컴퓨터 종류 등 이런 추가 정보가 요청 헤더에 포함되어 전달됨. 
    - 아래의 웹사이트에서 브라우저 헤더 확인 가능
        - http://myhttpheader.com/
- 위 첫 번째에서 얻은 URL을 사용하여 아마존 상품의 HTML 페이지를 요청 라이브러리를 사용하여 요청하기
    - 힌트1: 요청 시, 일부 헤더를 함께 전달해야 실제 웹사이트 HTML을 반환한다. 최소한 요청 헤더에 "User-Agent"와 "Accept-Language"값을 넣어야 한다. 
    - 힌트 2: 요청 라이브러리로 헤더를 전달하는 방법은 아래 링크를 참고
        - https://stackoverflow.com/questions/6260457/using-headers-with-the-python-requests-librarys-get-method
    - 힌트 3: 힌트 1에서 헤더를 더 많이 추가하지 않는다면, GET 요청의 결괏값을 출력하고, 웹페이지의 HTML이 실제로 출력되었는지 확인하기. 가끔씩, 로봇인지 묻는 CAPTCHA 페이지가 나오기도 한다.
- BeautifulSoup을 사용하여, 가져온 웹페이지 HTML로 soup 객체를 만드세요. ‘html.parser’ 대신 ‘lxml’ 파서를 사용해야 동작됨
    - 힌트: 만약 ‘bs4.FeatureNotFound: Couldn't find a tree builder with the features you requested: html-parser.’라는 오류가 뜬다면, 파서가 잘못됐다는 말이니, 제일 위에 lxml을 임포트하고 모듈을 설치한 뒤, soup 객체를 만들 때 ‘html.parser’ 대신 ‘lmxl’을 사용해야 한다.
-  BeautifulSoup을 사용하여 상품 가격을 부동 소수점 수로 구해서 출력하기
    - 힌트: split() 메소드를 사용해 보기

In [18]:
import requests
import lxml
from bs4 import BeautifulSoup

In [ ]:
# bs4 방식 
url = "https://www.amazon.com/SAMSUNG-Monitor-Streaming-Wireless-LS32BM703UNXZA/dp/B09Z2H2JJK/ref=sr_1_6?crid=38WPMAYUTCCU&keywords=samsung%2Btv&qid=1695124000&s=electronics&sprefix=samsung%2Btv%2Celectronics-intl-ship%2C248&sr=1-6&th=1"
headers = {"Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
          "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
response = requests.get(url, headers=headers)
amazon_info = response.content
soup = BeautifulSoup(amazon_info, "lxml")
print(soup.prettify())

price = soup.find(class_="a-offscreen").get_text()
price_without_currency = price.split("$")[1]
price_as_float = float(price_without_currency)
print(price_as_float)

- 여기서 아마존이 데이터가 불러와지지 않는다. 동적 스크래핑은 BeautifulSoup으로 어려움으로 셀레니움으로 하고자함

In [28]:
import requests
import lxml
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

import undetected_chromedriver as uc 
from selenium.webdriver.common.by import By    # 셀레니움 Element를 찾기 위한 모듈

# dirve option 
# fake user 사용 
us = UserAgent(verify_ssl=False)
userAgent = us.random
print(userAgent)

options = uc.ChromeOptions()

# 크롬드라이버 headless 옵션 
# options.headless=True
# options.add_argument('--headless')   
options.add_argument("--no-sandbox")  

options.add_argument(f'user-agent={userAgent}')  # fake user 설정 
options.add_argument("--load-images=yes")        # 이미지 로드 설정 
options.add_argument("--disable-extensions")     # 비활성화 - 어떤 것을 비활성화 하는지 잘 모르겠음 
options.add_argument("--disable-gpu")            # gpu 가속 사용 제외
options.add_argument("--lang=ja-JP")             # 가짜 플러그인 탑재 - 일본어 설정 
driver = uc.Chrome(options=options)              # 설정된 크롬드라이버 선언

# url 접속 
url = "https://www.amazon.com/SAMSUNG-Monitor-Streaming-Wireless-LS32BM703UNXZA/dp/B09Z2H2JJK/ref=sr_1_6?crid=38WPMAYUTCCU&keywords=samsung%2Btv&qid=1695124000&s=electronics&sprefix=samsung%2Btv%2Celectronics-intl-ship%2C248&sr=1-6&th=1"
driver.get(url)

# 셀레니움에 접속한 url 기준 page source 가져오기
page_source = driver.page_source

# lxml로 파싱하기
soup = BeautifulSoup(page_source, "lxml")

# 데이터 받기 
price = soup.find(class_="a-offscreen").get_text()
price_without_currency = price.split("$")[1]
price_as_float = float(price_without_currency)
print(price_as_float)

# 크롬드라이버 닫기 
driver.close()
driver.quit()

Mozilla/4.0 (compatible; MSIE 5.05; Windows NT 3.51)
348.0


### 2. 가격이 사전에 설정된 가격보다 낮을 때 이메일로 알려주기
- 상품 가격이 어느 선 이하로 떨어졌을 때, 이메일을 받으려 한다. 
    - 예) 삼성 스마트 모니터 경우, 280 달러로 설정.
- 가격이 280달러 이하로 떨어졌을 때, smtp 모듈을 사용하여 본인에게 이메일을 전송하기. 이메일에는 상품명, 현재 가격, 상품 구매 링크가 있어야 한다.
    - 힌트: 목표 가격을 현재 상품 가격보다 높게 설정하여 이메일 전송 테스트 하기

In [1]:
import requests
import lxml
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

import undetected_chromedriver as uc 
from selenium.webdriver.common.by import By    # 셀레니움 Element를 찾기 위한 모듈

In [2]:
# dirve option 
# fake user 사용 
us = UserAgent(verify_ssl=False)
userAgent = us.random
print(userAgent)

options = uc.ChromeOptions()

# 크롬드라이버 headless 옵션 
# options.headless=True
# options.add_argument('--headless')   
options.add_argument("--no-sandbox")  

options.add_argument(f'user-agent={userAgent}')  # fake user 설정 
options.add_argument("--load-images=yes")        # 이미지 로드 설정 
options.add_argument("--disable-extensions")     # 비활성화 - 어떤 것을 비활성화 하는지 잘 모르겠음 
options.add_argument("--disable-gpu")            # gpu 가속 사용 제외
options.add_argument("--lang=ja-JP")             # 가짜 플러그인 탑재 - 일본어 설정 
driver = uc.Chrome(options=options)              # 설정된 크롬드라이버 선언

# url 접속 
url = "https://www.amazon.com/SAMSUNG-Monitor-Streaming-Wireless-LS32BM703UNXZA/dp/B09Z2H2JJK/ref=sr_1_6?crid=38WPMAYUTCCU&keywords=samsung%2Btv&qid=1695124000&s=electronics&sprefix=samsung%2Btv%2Celectronics-intl-ship%2C248&sr=1-6&th=1"
driver.get(url)

# 셀레니움에 접속한 url 기준 page source 가져오기
page_source = driver.page_source

# lxml로 파싱하기
soup = BeautifulSoup(page_source, "lxml")

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14931


In [8]:
# 데이터 받기 
# 상품명
product = soup.find(class_="a-size-large product-title-word-break").get_text()
product = product.strip()
print(product)

# 가격
price = soup.find(class_="a-offscreen").get_text()
price_without_currency = price.split("$")[1]
price_as_float = float(price_without_currency)
print(price_as_float)

SAMSUNG 32" M70B Series 4K UHD USB-C Smart Monitor & Streaming TV, 4ms, 60Hz, HDR10, Wireless Display, Slimfit Camera, Gaming and IoT Hubs, Alexa Built in, 2022, LS32BM703UNXZA, White
399.05


In [9]:
# 크롬드라이버 닫기 
driver.close()
driver.quit()

In [10]:
import smtplib
from email.mime.text import MIMEText     # email 모듈 중 메시지 제목과 본문을 설정하기 위한 라이브러리
from email.mime.multipart import MIMEMultipart  # email 모듈 메일의 데이터 영역의 메시지를 만드는 라이브러리

In [11]:
# Email information
MY_EMAIL= "email"
GOOGLE_APP_PASSWORD = "password"
SEND_ADRESS_1 = "email address"

In [14]:
row_price_check = False
# 조건문 선언 
if price_as_float < 290:
    row_price_check = True

In [15]:
if row_price_check:
    
    # 메일 본문 작성 - html 형식 
    msg_text =  "안녕하세요.<br>" \
                "<br>" \
                "오늘 아침 9시에 아마존에서,<br>" \
                "<br>" \
                f"{product}를 검색해본 결과<br>" \
                "<br>" \
                "지정된 가격보다 낮아 메일 드립니다.<br>" \
                "<br>" \
                f"구매 링크:{url} 참조 부탁드립니다.<br>" \
                "<br>" \
                "지금 구매하세요!.<br>" \
                "<br>" \
                "가격 트레커 드림<br>" \
    
    # SMTP 서버 접속  
    connection = smtplib.SMTP("smtp.gmail.com", 587)                # SMTP 서버 연결 시작 
    connection.starttls()                                           # TLS 암호화 
    connection.login(user=MY_EMAIL, password=GOOGLE_APP_PASSWORD)   # SMTP 로그인
    
    # 수신인 이메일 주소 저장 - 여러 이메일 주소 
    recipients = [MY_EMAIL]
    
    str_from = MY_EMAIL                                              # 보내는 사람의 메일 주소 
    str_to = ", ".join(recipients)                                   # 받는 사람의 메일 주소 - 여러 메일 주소 설정 
    msg_root = MIMEMultipart("related")                              # 여러 MIME을 넣기위한 MIMEMultipart 객체 생성
    
    msg_root["Subject"] = "Hey Amazon!!!"                   # 메일 제목 설정 
    msg_root["From"] = str_from                                        # 보내는 사람 
    msg_root["To"] = str_to                                            # 받는 사람
    msg_alternative = MIMEMultipart('alternative')                     # 메일에 파일을 보내기 위한 MIMEMultipart 객체 생성
    msg_root.attach(msg_alternative)                                   # 선언된 MIMEMultipart 접근 
    msg = MIMEText(msg_text, 'html', _charset="utf8")                  # 메일 본문 내용 작성 
    msg_alternative.attach(msg)                                        # 작성된 메일 본문 접근 
    
    # 지정한 받는 사람 메일주소 별로 내용 전달 
    for recipient in recipients:
        connection.sendmail(
                            from_addr=str_from,                  # 보내는 사람
                            to_addrs=recipient,                  # 받는 사람 
                            msg=msg_root.as_string()             # 메세지 내용
                            )
    
    # SMTP 서버 종료 
    connection.quit()
else:
    # 메일 본문 작성 - html 형식 
    msg_text =  "안녕하세요.<br>" \
                "<br>" \
                "오늘 아침 9시에 아마존에서,<br>" \
                "<br>" \
                f"{product}를 검색해본 결과<br>" \
                "<br>" \
                "지정된 가격보다 아직 높습니다.<br>" \
                "<br>" \
                "감사합니다.<br>" \
                "<br>" \
                "가격 트레커 드림<br>" \
    
    # SMTP 서버 접속  
    connection = smtplib.SMTP("smtp.gmail.com", 587)                # SMTP 서버 연결 시작 
    connection.starttls()                                           # TLS 암호화 
    connection.login(user=MY_EMAIL, password=GOOGLE_APP_PASSWORD)   # SMTP 로그인
    
    # 수신인 이메일 주소 저장 - 여러 이메일 주소 
    recipients = [MY_EMAIL]
    
    str_from = MY_EMAIL                                              # 보내는 사람의 메일 주소 
    str_to = ", ".join(recipients)                                   # 받는 사람의 메일 주소 - 여러 메일 주소 설정 
    msg_root = MIMEMultipart("related")                              # 여러 MIME을 넣기위한 MIMEMultipart 객체 생성
    
    msg_root["Subject"] = "Hey Amazon!!!"                   # 메일 제목 설정 
    msg_root["From"] = str_from                                        # 보내는 사람 
    msg_root["To"] = str_to                                            # 받는 사람
    msg_alternative = MIMEMultipart('alternative')                     # 메일에 파일을 보내기 위한 MIMEMultipart 객체 생성
    msg_root.attach(msg_alternative)                                   # 선언된 MIMEMultipart 접근 
    msg = MIMEText(msg_text, 'html', _charset="utf8")                  # 메일 본문 내용 작성 
    msg_alternative.attach(msg)                                        # 작성된 메일 본문 접근 
    
    # 지정한 받는 사람 메일주소 별로 내용 전달 
    for recipient in recipients:
        connection.sendmail(
                            from_addr=str_from,                  # 보내는 사람
                            to_addrs=recipient,                  # 받는 사람 
                            msg=msg_root.as_string()             # 메세지 내용
                            )
    
    # SMTP 서버 종료 
    connection.quit()

### 전체코드

In [17]:
import requests
import lxml
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import undetected_chromedriver as uc 
from selenium.webdriver.common.by import By

import smtplib
from email.mime.text import MIMEText     # email 모듈 중 메시지 제목과 본문을 설정하기 위한 라이브러리
from email.mime.multipart import MIMEMultipart  # email 모듈 메일의 데이터 영역의 메시지를 만드는 라이브러리

# 상수 설정 
# Email information
MY_EMAIL= "email"
GOOGLE_APP_PASSWORD = "password"
SEND_ADRESS_1 = "email address"
# 저정가 지정 
SELECT_PRICE = 290

# dirve option 
# fake user 사용 
us = UserAgent(verify_ssl=False)
userAgent = us.random
print(userAgent)

options = uc.ChromeOptions()

# 크롬드라이버 headless 옵션 
# options.headless=True
# options.add_argument('--headless')   
options.add_argument("--no-sandbox")  

options.add_argument(f'user-agent={userAgent}')  # fake user 설정 
options.add_argument("--load-images=yes")        # 이미지 로드 설정 
options.add_argument("--disable-extensions")     # 비활성화 - 어떤 것을 비활성화 하는지 잘 모르겠음 
options.add_argument("--disable-gpu")            # gpu 가속 사용 제외
options.add_argument("--lang=ja-JP")             # 가짜 플러그인 탑재 - 일본어 설정 
driver = uc.Chrome(options=options)              # 설정된 크롬드라이버 선언

# url 접속 
url = "https://www.amazon.com/SAMSUNG-Monitor-Streaming-Wireless-LS32BM703UNXZA/dp/B09Z2H2JJK/ref=sr_1_6?crid=38WPMAYUTCCU&keywords=samsung%2Btv&qid=1695124000&s=electronics&sprefix=samsung%2Btv%2Celectronics-intl-ship%2C248&sr=1-6&th=1"
driver.get(url)

# 셀레니움에 접속한 url 기준 page source 가져오기
page_source = driver.page_source

# lxml로 파싱하기
soup = BeautifulSoup(page_source, "lxml")

# 데이터 받기 
# 상품명
product = soup.find(class_="a-size-large product-title-word-break").get_text()
product = product.strip()
print(product)

# 가격
price = soup.find(class_="a-offscreen").get_text()
price_without_currency = price.split("$")[1]
price_as_float = float(price_without_currency)
print(price_as_float)

# 크롬드라이버 닫기 
driver.close()
driver.quit()

# 메일 보내기

row_price_check = False

# 조건문 선언 
if price_as_float < SELECT_PRICE:
    row_price_check = True

# 만약 지정가 보다 저가일 경우, 지금 사라고 하는 메일 발송
if row_price_check:
    
    # 메일 본문 작성 - html 형식 
    msg_text =  "안녕하세요.<br>" \
                "<br>" \
                "오늘 아침 9시에 아마존에서,<br>" \
                "<br>" \
                f"{product}를 검색해본 결과<br>" \
                "<br>" \
                "지정된 가격보다 낮아 메일 드립니다.<br>" \
                "<br>" \
                f"구매 링크:{url} 참조 부탁드립니다.<br>" \
                "<br>" \
                "지금 구매하세요!.<br>" \
                "<br>" \
                "가격 트레커 드림<br>" \
    
    # SMTP 서버 접속  
    connection = smtplib.SMTP("smtp.gmail.com", 587)                # SMTP 서버 연결 시작 
    connection.starttls()                                           # TLS 암호화 
    connection.login(user=MY_EMAIL, password=GOOGLE_APP_PASSWORD)   # SMTP 로그인
    
    # 수신인 이메일 주소 저장 - 여러 이메일 주소 
    recipients = [MY_EMAIL]
    
    str_from = MY_EMAIL                                              # 보내는 사람의 메일 주소 
    str_to = ", ".join(recipients)                                   # 받는 사람의 메일 주소 - 여러 메일 주소 설정 
    msg_root = MIMEMultipart("related")                              # 여러 MIME을 넣기위한 MIMEMultipart 객체 생성
    
    msg_root["Subject"] = "Hey Amazon!!!"                   # 메일 제목 설정 
    msg_root["From"] = str_from                                        # 보내는 사람 
    msg_root["To"] = str_to                                            # 받는 사람
    msg_alternative = MIMEMultipart('alternative')                     # 메일에 파일을 보내기 위한 MIMEMultipart 객체 생성
    msg_root.attach(msg_alternative)                                   # 선언된 MIMEMultipart 접근 
    msg = MIMEText(msg_text, 'html', _charset="utf8")                  # 메일 본문 내용 작성 
    msg_alternative.attach(msg)                                        # 작성된 메일 본문 접근 
    
    # 지정한 받는 사람 메일주소 별로 내용 전달 
    for recipient in recipients:
        connection.sendmail(
                            from_addr=str_from,                  # 보내는 사람
                            to_addrs=recipient,                  # 받는 사람 
                            msg=msg_root.as_string()             # 메세지 내용
                            )
    
    # SMTP 서버 종료 
    connection.quit()

# 만약 지정가 보다 높다면, 다음에 구매해라라는 메일 보내기 
else:
    # 메일 본문 작성 - html 형식 
    msg_text =  "안녕하세요.<br>" \
                "<br>" \
                "오늘 아침 9시에 아마존에서,<br>" \
                "<br>" \
                f"{product}를 검색해본 결과<br>" \
                "<br>" \
                "지정된 가격보다 아직 높습니다.<br>" \
                "<br>" \
                "감사합니다.<br>" \
                "<br>" \
                "가격 트레커 드림<br>" \
    
    # SMTP 서버 접속  
    connection = smtplib.SMTP("smtp.gmail.com", 587)                # SMTP 서버 연결 시작 
    connection.starttls()                                           # TLS 암호화 
    connection.login(user=MY_EMAIL, password=GOOGLE_APP_PASSWORD)   # SMTP 로그인
    
    # 수신인 이메일 주소 저장 - 여러 이메일 주소 
    recipients = [MY_EMAIL]
    
    str_from = MY_EMAIL                                              # 보내는 사람의 메일 주소 
    str_to = ", ".join(recipients)                                   # 받는 사람의 메일 주소 - 여러 메일 주소 설정 
    msg_root = MIMEMultipart("related")                              # 여러 MIME을 넣기위한 MIMEMultipart 객체 생성
    
    msg_root["Subject"] = "Hey Amazon!!!"                   # 메일 제목 설정 
    msg_root["From"] = str_from                                        # 보내는 사람 
    msg_root["To"] = str_to                                            # 받는 사람
    msg_alternative = MIMEMultipart('alternative')                     # 메일에 파일을 보내기 위한 MIMEMultipart 객체 생성
    msg_root.attach(msg_alternative)                                   # 선언된 MIMEMultipart 접근 
    msg = MIMEText(msg_text, 'html', _charset="utf8")                  # 메일 본문 내용 작성 
    msg_alternative.attach(msg)                                        # 작성된 메일 본문 접근 
    
    # 지정한 받는 사람 메일주소 별로 내용 전달 
    for recipient in recipients:
        connection.sendmail(
                            from_addr=str_from,                  # 보내는 사람
                            to_addrs=recipient,                  # 받는 사람 
                            msg=msg_root.as_string()             # 메세지 내용
                            )
    
    # SMTP 서버 종료 
    connection.quit()

Mozilla/4.0 (compatible; MSIE 6.0; Windows 98; en) Opera 8.52
SAMSUNG 32" M70B Series 4K UHD USB-C Smart Monitor & Streaming TV, 4ms, 60Hz, HDR10, Wireless Display, Slimfit Camera, Gaming and IoT Hubs, Alexa Built in, 2022, LS32BM703UNXZA, White
399.05
